## RNN tutorial

This tutorial discusses how to develop RNN's in PyTorch, in particular RNNs for time series prediction. The following topics are covered:
* Tensor structure for sequences
* Vanilla RNN model in Pytorch
* Stacked RNNs
* Predicting n-day values

The data used in the tutorial can be downloaded from: https://www.kaggle.com/datasets/sumanthvrao/daily-climate-time-series-data

__NOTE__: The code in this tutorial is sometimes unecessarily verbose to expose students to coding practices that are often helpful in developing machine learning pipelines. Where this is the case, the code will be marked with "# verbose code"

Connect environment to a GPU by:
* Select 'Runtime' in the top left
* Select 'Change Runtime Type'
* Select the GPU runtime available

In [1]:
import wandb
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
from typing import Union, Callable, Tuple, List, Literal, Dict
from torch.autograd import Variable
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import logging
from abc import ABCMeta, abstractmethod
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### Understanding Tensor Structure for Sequences
When working with sequences, such as time series or sentences, RNNs require input data to be formatted as tensors with specific dimensions. Understanding these tensor dimensions is crucial for developing RNN models.

#### Tensor Dimensions in RNNs
RNNs in PyTorch expect input tensors in a specific format. For a batch of sequences, the input tensor dimensions are:

* Batch Size (`N`)**: The number of sequences in a batch.
* Sequence Length (`T`)**: The length of each sequence (e.g., number of time steps in a time series).
* Feature Size (`F`)**: The dimensionality of each input element (e.g., the number of features for each time step).

Thus, the input tensor shape is `(N, T, F)` when `batch_first=True`. Otherwise, it is `(T, N, F)`.


In [2]:
# Example: Batch size of 2, sequence length of 5, feature size of 3
batch_size = 2
seq_length = 5
feature_size = 3

# Create a random tensor with shape (batch_size, seq_length, feature_size)
input_tensor = torch.randn(batch_size, seq_length, feature_size)
print("Input Tensor Shape:", input_tensor.shape)  # Output: (2, 5, 3)

Input Tensor Shape: torch.Size([2, 5, 3])


#### Flow of Tensors Through RNN Models
* Input to RNN Layer: The input tensor of shape (N, T, F) is fed into the RNN layer. The RNN processes each element in the sequence one time step at a time, using its internal hidden state, which is updated after processing each element.
* Hidden State Initialization: At the beginning of the sequence processing, the hidden state is initialized (usually to zeros). The shape of the hidden state tensor is (num_layers * num_directions, N, hidden_size), where:
  * num_layers is the number of RNN layers.
  * num_directions is 2 if the RNN is bidirectional; otherwise, it is 1.
  * hidden_size is the number of features in the hidden state.
* Output of RNN Layer:
  * Output Tensor: Shape (N, T, hidden_size) if batch_first=True. It contains the hidden states from the last RNN layer for each time step in the sequence.
  * Hidden State Tensor: Shape (num_layers * num_directions, N, hidden_size), which contains the hidden state for the last time step of each layer. The number of hidden state is a hyperparameter.
* Output to Fully Connected Layer:
  * In many applications, we only care about the output at the final time step. Therefore, we often take the output tensor from the last time step (out[:, -1, :] if batch_first=True) and pass it to a fully connected layer to make the final prediction.

In [3]:
# RNN configuration
input_size = feature_size  # Number of input features per time step
hidden_size = 4  # Number of features in the hidden state
num_layers = 1  # Number of stacked RNN layers

# Define the RNN layer
rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)

# Initialize hidden state
h0 = torch.zeros(num_layers, batch_size, hidden_size)

# Forward pass through the RNN
output, hn = rnn(input_tensor, h0)
print("Output Tensor Shape:", output.shape)  # Output: (2, 5, 4)
print("Hidden State Shape:", hn.shape)  # Output: (1, 2, 4)


Output Tensor Shape: torch.Size([2, 5, 4])
Hidden State Shape: torch.Size([1, 2, 4])


##### Using RNNs with Differing Sequence Length Inputs
RNNs expect inputs to be in batches with a consistent sequence length for each batch. However, in many real-world scenarios, sequences have differing lengths. For instance:

* In text processing, sentences can have different numbers of words.
* In time-series data, different samples may have varying numbers of time steps.
Handling these varying lengths in RNNs requires special techniques to ensure that shorter sequences are correctly processed without losing their shorter time-step information.

##### Approaches to Handling Varying Sequence Lengths in RNNs
* Padding Sequences: The most common approach to dealing with varying sequence lengths is to pad the shorter sequences so that all sequences in a batch have the same length. Padding is achieved by adding dummy values (typically zeros) to the end of shorter sequences. The padded sequences can then be processed by the RNN, but the padding itself should not affect the learning process. We can address this by using masking or packing techniques.
* Packed sequences allow the RNN to efficiently ignore the padding by only processing the actual (non-padded) sequence lengths.

In [4]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# Example input sequences with different lengths, each time step has 3 features
sequences = [
    torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),  # length 3 (3 time steps, each with 3 features)
    torch.tensor([[10, 11, 12], [13, 14, 15]]),       # length 2
    torch.tensor([[16, 17, 18], [19, 20, 21], [22, 23, 24], [25, 26, 27]])  # length 4
]

# Step 1: Pad the sequences to the same length
padded_sequences = pad_sequence(sequences, batch_first=True)

# Example sequence lengths
lengths = torch.tensor([3, 2, 4])

print("Padded sequences:\n", padded_sequences)

# Step 2: Define RNN
input_size = 3  # Number of features in each time step
hidden_size = 4  # Number of features in the hidden state
batch_size = len(sequences)

rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
# Since the input_size is 1, we need to add an extra dimension to the input
padded_sequences = padded_sequences.float()

# Step 3: Pack the padded sequences
packed_input = pack_padded_sequence(padded_sequences, lengths, batch_first=True, enforce_sorted=False)

# Step 4: Pass through RNN
h0 = torch.zeros(1, batch_size, hidden_size).float()  # Initialize hidden state as float
packed_output, hn = rnn(packed_input, h0)

# Step 5: Unpack the output
output, output_lengths = pad_packed_sequence(packed_output, batch_first=True)

print("Packed output shape:", output.shape)  # Shape: (batch_size, max_seq_len, hidden_size)
print("Hidden state shape:", hn.shape)        # Shape: (1, batch_size, hidden_size)

Padded sequences:
 tensor([[[ 1,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9],
         [ 0,  0,  0]],

        [[10, 11, 12],
         [13, 14, 15],
         [ 0,  0,  0],
         [ 0,  0,  0]],

        [[16, 17, 18],
         [19, 20, 21],
         [22, 23, 24],
         [25, 26, 27]]])
Packed output shape: torch.Size([3, 4, 4])
Hidden state shape: torch.Size([1, 3, 4])


##### Explanation of the above example code
* Input Tensor: The input tensor is of shape (2, 5, 3), which matches the required (N, T, F) format.
* RNN Output: The output tensor has the shape (2, 5, 4), which means for each of the 2 sequences (batch size), there are 5 time steps, and the hidden state has 4 features.
* Hidden State (hn): The hidden state tensor at the last time step has the shape (1, 2, 4), representing 1 layer, 2 sequences, and a hidden state size of 4.

In [5]:
data_dir = '/cs/academic/phd3/xinrzhen/xinran/TA'
logger = logging.getLogger("rnn_tutorial")
logger.setLevel(logging.INFO)
WANDB_PROJ = "rnn_tutorial"
TMP_DIR = "./tmp"

if os.path.isdir(TMP_DIR):
    pass
else:
    os.mkdir(TMP_DIR)

#### Prepare dataset
The double for loop in the code block (also copied directly) below defines the train/test data structure (mentioned above):
| date | meantemp at time t | humidity at time t | wind_speed at time t | meanpressure at time t | meantemp at time t+1 |
| ---- | ---- | ---- | ---- | ---- | ---- |
| .... | .... | .... | .... | .... | .... | 

* Generate datasets for different steps
  * Setting a step causes columns with a step greater than 1 to be shorter than the original DataFrame. This is because the shift(-step) function shifts the data up a step row, which causes the last few rows of the data to become NaN when the step is greater than 1. 
  * These NaN values are usually discarded when dealing with prediction problems, reducing the amount of valid data.
```python
steps = [1,5,10]
for df in [train_df, test_df]:
    for stp in steps:
        df[[f"{col}_{stp}_step" for col in non_date_vars]] = df[non_date_vars].shift(-1*stp)
```
By resetting dataset following differents steps, 

However, it is generalised in the following ways:
* To compute not just the meantemp at t+1 but all features at time t+1;
* To compute t+step target values, not just t+1

This generalisation is useful for performing __Exercise 6b__ and __Exercise 8__.


In [6]:
train_df = pd.read_csv(os.path.join(data_dir, "DailyDelhiClimateTrain.csv"))
test_df = pd.read_csv(os.path.join(data_dir, "DailyDelhiClimateTest.csv"))

train_df["date"] = pd.to_datetime(train_df["date"], format="%Y-%m-%d")
print(train_df.head())

        date   meantemp   humidity  wind_speed  meanpressure
0 2013-01-01  10.000000  84.500000    0.000000   1015.666667
1 2013-01-02   7.400000  92.000000    2.980000   1017.800000
2 2013-01-03   7.166667  87.000000    4.633333   1018.666667
3 2013-01-04   8.666667  71.333333    1.233333   1017.166667
4 2013-01-05   6.000000  86.833333    3.700000   1016.500000


In [7]:
# Prepare the dataset with multiple step predictions
non_date_vars = [col for col in train_df.columns if col != "date"]  # Exclude date column
steps = [1, 5, 10]  # Predict future values at different steps
for df in [train_df, test_df]:
    for step in steps:
        df[[f"{col}_{step}_step" for col in non_date_vars]] = df[non_date_vars].shift(-step)

print(train_df.head())
print(train_df.tail())  

        date   meantemp   humidity  wind_speed  meanpressure  meantemp_1_step  \
0 2013-01-01  10.000000  84.500000    0.000000   1015.666667         7.400000   
1 2013-01-02   7.400000  92.000000    2.980000   1017.800000         7.166667   
2 2013-01-03   7.166667  87.000000    4.633333   1018.666667         8.666667   
3 2013-01-04   8.666667  71.333333    1.233333   1017.166667         6.000000   
4 2013-01-05   6.000000  86.833333    3.700000   1016.500000         7.000000   

   humidity_1_step  wind_speed_1_step  meanpressure_1_step  meantemp_5_step  \
0        92.000000           2.980000          1017.800000         7.000000   
1        87.000000           4.633333          1018.666667         7.000000   
2        71.333333           1.233333          1017.166667         8.857143   
3        86.833333           3.700000          1016.500000        14.000000   
4        82.800000           1.480000          1018.000000        11.000000   

   humidity_5_step  wind_speed_5_step 

#### Train/test split
* The training dataset also needs to be split into a training and holdout set. When using any data where observations are non iid, data must be split to prevent "data leakage". Time series data is likely non-iid in the sense that future observations most likely depend on previous ones for example, it is reasonable to assume that the meantemp at time t+1 is dependant on the meantemp at time t.
* The dataset therefore needs to be split such that the ML models is not explosed to correlations which would not be available at test time.
* Given this, the final year of the training data is used as the holdout set

In [8]:
# Split data into training and holdout sets to prevent data leakage
train_df["__date_yrs"] = train_df["date"].dt.year
val_idx = train_df["__date_yrs"] >= 2016  # Holdout data for the last year
val_df = train_df[val_idx].drop(columns=[col for col in train_df.columns if col.startswith("__")])
train_df = train_df[~val_idx].drop(columns=[col for col in train_df.columns if col.startswith("__")])
print(train_df.shape, val_df.shape, test_df.shape)
train_df.head()

(1095, 17) (367, 17) (114, 17)


,date,meantemp,humidity,wind_speed,meanpressure,meantemp_1_step,humidity_1_step,wind_speed_1_step,meanpressure_1_step,meantemp_5_step,humidity_5_step,wind_speed_5_step,meanpressure_5_step,meantemp_10_step,humidity_10_step,wind_speed_10_step,meanpressure_10_step
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667,7.400000,92.000000,2.980000,1017.800000,7.000000,82.800000,1.480000,1018.000000,15.714286,51.285714,10.571429,1016.142857
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000,7.166667,87.000000,4.633333,1018.666667,7.000000,78.600000,6.300000,1020.000000,14.000000,74.000000,13.228571,1015.571429
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667,8.666667,71.333333,1.233333,1017.166667,8.857143,63.714286,7.142857,1018.714286,15.833333,75.166667,4.633333,1013.333333
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667,6.000000,86.833333,3.700000,1016.500000,14.000000,51.250000,12.500000,1017.000000,12.833333,88.166667,0.616667,1015.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000,7.000000,82.800000,1.480000,1018.000000,11.000000,62.000000,7.400000,1015.666667,14.714286,71.857143,0.528571,1015.857143


### Predicting next day values
* For the first exercise, daily climate data will be used to develop a model that can predict the next day "meantemp".
* First of all, training and test datasets need to be manipulated such that they are in the following form:
| date | meantemp at time t | humidity at time t | wind_speed at time t | meanpressure at time t | meantemp at time t+1 |
| ---- | ---- | ---- | ---- | ---- | ---- |
| .... | .... | .... | .... | .... | .... | 

##### Implementing the Vanilla RNN Model in PyTorch
We will now implement a simple RNN model using PyTorch's nn.RNN module. This model will take sequences of historical data and predict future values.

* Model Architecture
  * RNN Layer: The core component of the model that processes sequences.
  * Fully Connected Layer: A linear layer that maps the hidden states produced by the RNN to the output space.

In [9]:
# verbose code
# Here a series of Debug classes are defined. The reason for using this structure will be discussed in class
class DebugPass:

    def __init__(self):
        self.y_pred = []
        self.y_true = []

    def debug(
        self, 
        y_true:torch.tensor, 
        y_pred:torch.tensor, 
    ):
        pass
        
    def close(self):
        pass

class DebugBase(DebugPass):

    def __init__(self):
        super().__init__()

    def debug(self, y_true, y_pred):
        self.y_true.append(y_true.cpu().detach().numpy())
        self.y_pred.append(y_pred.cpu().detach().numpy())

    @abstractmethod
    def close(self):
        pass

class DebugLocal(DebugBase):

    def __init__(self):
        super().__init__()

    def close(self):
        res_tbl = pd.DataFrame(
            {
                "y_true":np.concatenate(self.y_true, axis=0).squeeze().flatten(), 
                "y_pred":np.concatenate(self.y_pred, axis=0).squeeze().flatten()
            }
        )
        res_tbl.to_csv(os.path.join(TMP_DIR, "validate_debug.csv"), index=False)

class DebugWandB(DebugBase):

    def __init__(self):
        super().__init__()

    def close(self):
        res_tbl = pd.DataFrame(
            {
                "y_true":np.concatenate(self.y_true, axis=0).squeeze().flatten(), 
                "y_pred":np.concatenate(self.y_pred, axis=0).squeeze().flatten()
            }
        )
        wandb_tbl = wandb.Table(dataframe=res_tbl)
        wandb.log({"val_predictions" : wandb_tbl})

def train_single_epoch(model:nn.Module, data_loader:torch.utils.data.DataLoader, 
                       gpu:Literal[True, False], optimizer:torch.optim,
                       criterion:torch.nn.modules.loss
                      ) -> Tuple[List[torch.Tensor]]:
    model.train()
    model.to(device)
    losses = []
    preds = []
    range_gen = tqdm(
        enumerate(data_loader),
        )
    for i, (y,X) in range_gen:
        
        X = X.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()

        # Compute output
        output = model(X)
        preds.append(output)
        
        train_loss = criterion(output, y)
        losses.append(train_loss.item())

        # losses.update(train_loss.data[0], g.size(0))
        # error_ratio.update(evaluation(output, target).data[0], g.size(0))

        try: 
            # compute gradient and do SGD step
            train_loss.backward()
            
            optimizer.step()
        except RuntimeError as e:
            print("Runtime error on training instance: {}".format(i))
            raise e
    return losses, preds

def validate(model:nn.Module, data_loader:torch.utils.data.DataLoader,
             gpu:Literal[True, False], criterion:torch.nn.modules.loss,
             dh:DebugPass
            ) -> Tuple[List[torch.Tensor]]:
    
    model.eval()
    model.to(device)
    losses = []
    preds = []
    with torch.no_grad():
        range_gen = tqdm(
            enumerate(data_loader),
        )
        # Your code here
        for i, (y,X) in range_gen:
        
            X = X.to(device)
            y = y.to(device)

            # Compute output
            output = model(X)

            # Logs
            losses.append(criterion(output, y).item())
            preds.append(output)
            dh.debug(y_true=y, y_pred=output)
    return losses, preds


def train(model:torch.nn, train_data_loader:torch.utils.data.DataLoader,
          val_data_loader:torch.utils.data.DataLoader, 
          gpu:Literal[True, False], optimizer:torch.optim,
          criterion:torch.nn.modules.loss, epochs:int, 
          debug:bool = False, wandb_proj:str="", 
          wandb_config:Dict={}
         ) -> Tuple[List[torch.Tensor]]:

    if (len(wandb_config) == 0) or (len(wandb_proj) == 0):
        use_wandb = False
        logger.warning("WandB not in use!")
        chkpnt_dir = TMP_DIR
    else:
        use_wandb = True
        wandb.init(project=wandb_proj, config=wandb_config)
        chkpnt_dir = wandb.run.dir

    if debug:
        if use_wandb:
            dh = DebugWandB()
        else:
            dh = DebugLocal()
    else:
        dh = DebugPass()
    
    if gpu:
        model.cuda()
    
    epoch_train_loss = []
    epoch_val_loss = []
    for epoch in range(1, epochs+1):
        print("Running training epoch")
        train_loss_val, train_preds =  train_single_epoch(
            model=model, data_loader=train_data_loader, gpu=gpu, 
            optimizer=optimizer, criterion=criterion)
        mean_train_loss = np.mean(train_loss_val)
        epoch_train_loss.append(mean_train_loss)
        val_loss_val, val_preds = validate(
            model=model, data_loader=val_data_loader, gpu=gpu, 
            criterion=criterion, dh=dh)
        
        print("Running validation")
        mean_val_loss = np.mean(val_loss_val)
        epoch_val_loss.append(np.mean(val_loss_val))

        chkp_pth = os.path.join(chkpnt_dir, f"mdl_chkpnt_epoch_{epoch}.pt")
        torch.save(
            {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, chkp_pth)
        if use_wandb:
            wandb.log({"train_loss": mean_train_loss, "val_loss": mean_val_loss})
            wandb.save(chkp_pth)
    dh.close()
    if use_wandb: 
        wandb.finish()
    return epoch_train_loss, epoch_val_loss

##### Create Dataset class for time series data format
* Using the PandasDataset class, data is batched according to the time dimension. This dataset object needs to be extended such that sequences of length longer than 1 can be created.


In [10]:
class PandasDataset(Dataset):
    def __init__(self, X:pd.DataFrame, y:pd.Series, normalise:bool=True)->None:
        self._X = torch.from_numpy(X.values).float()
        if normalise:
            self._X = self.__min_max_norm(self._X)
        self.feature_dim = X.shape[1]
        self._len = X.shape[0]
        self._y = torch.from_numpy(y.values)[:,None].float()
    
    def __len__(self)->int:
        return self._len
    
    def __getitem__(self, idx:int) -> Tuple[torch.Tensor, torch.Tensor]:
        return self._y[idx], self._X[idx,:]
        
    def __min_max_norm(self, in_tens:torch.Tensor) -> torch.Tensor:
        # X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
        _min = in_tens.min(axis=0).values
        _max = in_tens.max(axis=0).values
        in_tens = (in_tens - _min)/(_max - _min)
        return in_tens
    

#### Predict 1 step mean temperature
* For the first exercise, the aim is to predict the next day mean temperature using a historial time series of temperature, humidity, wind_speed and meanpressure values
* A core hyperparameter is defining the sequence length i.e., the size of the historial time series to use for prediction.
* As it stands, the data is of the form (time t obs, time t+1 target). Therefore, if this data was converted to a tensor as is, batched and used for training, only the time t values would be used for prediction. Using the "PandasDataset" class from the first tutorial demonstrates this

In [11]:
trgt_col = "meantemp_1_step" # This is the t+1 target associated with the time t observations
# Dynamically select the remaining feature columns i.e., those that are: 
# 1) Not target variables (do not end with _step) and;
# 2) Are not the date variable
indp_cols = [ # verbose code
    col for col in train_df.columns if (
        (col != "date") and (col[-5:] != "_step")
    )
]
print(trgt_col)
print(indp_cols)

meantemp_1_step
['meantemp', 'humidity', 'wind_speed', 'meanpressure']


In [12]:
# Normalise has been set to False for demo purposes
tmp_dataset = PandasDataset(X=train_df[indp_cols], y=train_df[trgt_col], normalise=False)
tmp_loader = DataLoader(tmp_dataset, shuffle=False, batch_size=2)
display(train_df[indp_cols+[trgt_col]].head(4))
loader_iter = tmp_loader.__iter__()
first_batch = next(loader_iter)
print(f"The first batch contains the first two rows of the dataset:\n {first_batch[1]}")
second_batch = next(loader_iter)
print(f"The second batch contains the second two rows of the dataset:\n {second_batch[1]}")

,meantemp,humidity,wind_speed,meanpressure,meantemp_1_step
0,10.000000,84.500000,0.000000,1015.666667,7.400000
1,7.400000,92.000000,2.980000,1017.800000,7.166667
2,7.166667,87.000000,4.633333,1018.666667,8.666667
3,8.666667,71.333333,1.233333,1017.166667,6.000000


The first batch contains the first two rows of the dataset:
 tensor([[  10.0000,   84.5000,    0.0000, 1015.6667],
        [   7.4000,   92.0000,    2.9800, 1017.8000]])
The second batch contains the second two rows of the dataset:
 tensor([[   7.1667,   87.0000,    4.6333, 1018.6667],
        [   8.6667,   71.3333,    1.2333, 1017.1667]])


__Exercise 1a__: 
* The __get_lookback function is designed to augment the _X and _y tensors with sequences of length "lookback".
* Where lookback is defined as 2, feature values at time points 't' and 't-1' are required to predict values at timepoint 't+1'.
* The code contains a bug where the dimensions of the _X and _y are incorrect - fix this

__Exercise 1b__: 
* Consider why the target tensor is indexed as follows ```i+lookback-1:i+lookback```

In [12]:
class PandasTsDataset(PandasDataset):
    def __init__(self, X: pd.DataFrame, y: pd.Series, lookback: int, normalise: bool = True) -> None:
        super().__init__(X=X, y=y, normalise=normalise)
        if lookback > 1:
            self.__get_lookback(lookback=lookback)
        # Update the length to reflect the lookback window
        self._len = self._X.shape[0]
    
    def __get_lookback(self, lookback: int):
        X_vals = []
        y_vals = []
        # Ensure we stop correctly to avoid index out of bounds
        for i in range(self._X.shape[0] - lookback + 1):
            X_vals.append(self._X[i:i + lookback].unsqueeze(0))  # Add an extra dimension
            y_vals.append(self._y[i + lookback - 1])  # Use the last value in the lookback period as the target
        self._X = torch.cat(X_vals, axis=0)
        self._y = torch.stack(y_vals, axis=0)
    
    def __len__(self) -> int:
        return self._len
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        # Return feature and target tensors for the given index
        return self._y[idx], self._X[idx]

In [14]:
tmp_dataset = PandasTsDataset(X=train_df[indp_cols], y=train_df[trgt_col], lookback=2, normalise=False)
display(train_df.head())
print(f"First row of the data contains the first and second row of the original dataset:\n {tmp_dataset[0][1]}")
print(f"With the target defined as:\n {tmp_dataset[0][0]}")
print("\n")
print(f"Second row of the data contains the second and third row of the original dataset:\n {tmp_dataset[1][1]}")
print(f"With the target defined as:\n {tmp_dataset[1][0]}")
print("\n")
print(f"Final row of the data contains the penultimate and final row of the original dataset:\n {tmp_dataset[-1][1]}")
print(f"With the target defined as:\n {tmp_dataset[-1][0]}")
display(train_df.tail())
print("\n")
print("\n")
tmp_loader = DataLoader(tmp_dataset, shuffle=False, batch_size=2)
first_batch = next(tmp_loader.__iter__())
print(f"The first batch contains input:\n {first_batch[1]}")
print(f"With target values:\n {first_batch[0]}")

,date,meantemp,humidity,wind_speed,meanpressure,meantemp_1_step,humidity_1_step,wind_speed_1_step,meanpressure_1_step,meantemp_5_step,humidity_5_step,wind_speed_5_step,meanpressure_5_step,meantemp_10_step,humidity_10_step,wind_speed_10_step,meanpressure_10_step
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667,7.400000,92.000000,2.980000,1017.800000,7.000000,82.800000,1.480000,1018.000000,15.714286,51.285714,10.571429,1016.142857
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000,7.166667,87.000000,4.633333,1018.666667,7.000000,78.600000,6.300000,1020.000000,14.000000,74.000000,13.228571,1015.571429
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667,8.666667,71.333333,1.233333,1017.166667,8.857143,63.714286,7.142857,1018.714286,15.833333,75.166667,4.633333,1013.333333
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667,6.000000,86.833333,3.700000,1016.500000,14.000000,51.250000,12.500000,1017.000000,12.833333,88.166667,0.616667,1015.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000,7.000000,82.800000,1.480000,1018.000000,11.000000,62.000000,7.400000,1015.666667,14.714286,71.857143,0.528571,1015.857143


First row of the data contains the first and second row of the original dataset:
 tensor([[  10.0000,   84.5000,    0.0000, 1015.6667],
        [   7.4000,   92.0000,    2.9800, 1017.8000]])
With the target defined as:
 tensor([7.1667])


Second row of the data contains the second and third row of the original dataset:
 tensor([[   7.4000,   92.0000,    2.9800, 1017.8000],
        [   7.1667,   87.0000,    4.6333, 1018.6667]])
With the target defined as:
 tensor([8.6667])


Final row of the data contains the penultimate and final row of the original dataset:
 tensor([[  15.5000,   71.7500,    2.1000, 1017.5000],
        [  15.0000,   71.3750,    2.0875, 1020.5000]])
With the target defined as:
 tensor([14.7143])


,date,meantemp,humidity,wind_speed,meanpressure,meantemp_1_step,humidity_1_step,wind_speed_1_step,meanpressure_1_step,meantemp_5_step,humidity_5_step,wind_speed_5_step,meanpressure_5_step,meantemp_10_step,humidity_10_step,wind_speed_10_step,meanpressure_10_step
1090,2015-12-27,15.375,63.250,7.8875,1020.625,17.125000,58.125000,10.887500,1020.875000,14.714286,72.285714,1.057143,1021.142857,17.375000,81.625000,2.312500,1016.500000
1091,2015-12-28,17.125,58.125,10.8875,1020.875,16.375000,65.000000,7.412500,1018.125000,14.000000,75.875000,2.087500,1021.000000,17.125000,87.000000,0.000000,1018.125000
1092,2015-12-29,16.375,65.000,7.4125,1018.125,15.500000,71.750000,2.100000,1017.500000,14.375000,74.750000,5.112500,1018.500000,15.500000,83.250000,7.887500,1017.250000
1093,2015-12-30,15.500,71.750,2.1000,1017.500,15.000000,71.375000,2.087500,1020.500000,15.750000,77.125000,0.000000,1017.625000,15.857143,65.142857,8.471429,1015.428571
1094,2015-12-31,15.000,71.375,2.0875,1020.500,14.714286,72.285714,1.057143,1021.142857,15.833333,88.833333,0.616667,1017.000000,15.625000,74.375000,2.775000,1017.500000






The first batch contains input:
 tensor([[[  10.0000,   84.5000,    0.0000, 1015.6667],
         [   7.4000,   92.0000,    2.9800, 1017.8000]],

        [[   7.4000,   92.0000,    2.9800, 1017.8000],
         [   7.1667,   87.0000,    4.6333, 1018.6667]]])
With target values:
 tensor([[7.1667],
        [8.6667]])


The RNN model is now ready to be defined. To begin with, we'll try just using the nn.RNN module, provided by Pytorch. Consider the picture of an RNN below (credit: https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks):

![alternative text](/cs/academic/phd3/xinrzhen/xinran/TA/figures/generic_rnn_term_pred.png)

The blue blocks represent a single RNN computation and each computation take a set of hidden values, $a^{<t-1>}$, an input $x^{<t>}$ and produces a hidden state itself $a^{<t>}$. The final computation in the sequence produces an output, $y$.

* Mapping these to the input parameters of nn.RNN, 
    * input_size: Represents the dimenion of $x^{<t>}$ i.e., this represents the feature dimension for each observation in the input sequence
    * hidden_size: Represents the dimension of the hidden layer within the RNN
    * num_layers: Represents the number of "stacked" RNNs. Note this __does not__ represent the number of RNN computations. Ignore this parameter for now it is discussed later
    * nonlinearity: Represents the non-linear function which produces the set of hidden values $a^{<t>}$
    * batch_first: If set to True, tells the RNN to expect tensors of dimension (batch_size, sequence_size, feature_size) else it expects (sequence_size, batch_size, feature_size)
    * bidirectional: If set to true a 'bidirectional' RNN is defined. This is out of scope for the tutorial

The computation described by a single RNN unit is defined by:
\begin{equation}
    a^{<t>} = \textrm{nonlinearity}(x^{<t>}W_{i,h} + b_{i,h} + a^{<t-1>}W_{h,h} + b_{h,h})
\end{equation}

__Exercise__ 2: The computation described in https://pytorch.org/docs/stable/generated/torch.nn.RNN.html and https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks are identical (however, the $b_{i,h}$ bias is set to the identity in the cheatsheet). Try to reconclie the two with pen and paper.  

__Exercise__ 3:
* Using the description above, set the parameters below assuming we require:
    * Sequences of length 2 for each input
    * The dimension of $W_{h,h}$ to be 52
    * relu activation functions
    * Whether the input data should be shuffled

In [16]:
# Define parameters
lookback = 2
input_dim = len(indp_cols)
hidden_dim = 52
nonlinearity = "relu"
shuffle=True
num_layers = 1
fc_output_size = 1  # Output size for regression task

# Create datasets
train_dataset = PandasTsDataset(X=train_df[indp_cols], y=train_df[trgt_col], lookback=lookback)
val_dataset = PandasTsDataset(X=val_df[indp_cols], y=val_df[trgt_col], lookback=lookback)
print(len(train_dataset))
print(train_df[indp_cols].shape[0])
train_loader = DataLoader(dataset=train_dataset, shuffle=shuffle, batch_size=2)
val_loader = DataLoader(dataset=val_dataset, shuffle=shuffle, batch_size=2)

print(f"Train Dataset: {len(train_dataset)} samples")
print(f"Validation Dataset: {len(val_dataset)} samples")


1094
1095
Train Dataset: 1094 samples
Validation Dataset: 366 samples


__Exercise__ 4:
* The code below produces a bug. Debug it and define the VanillaRNN class
* _Hints_:
    * Performing the reconciliation exercise will help with this, in particular noticing that the description in Pytorch (and therefore the computation implemented in the nn.RNN function is __missing__ the computation $y = g_{2}(W_{y,a}a^{<T>} + b_{y})$) where $a^{<T>}$ is the hidden layer output from the final RNN computation
    * Also, examine the object type produced by the RNN() call. Is it what you expect? Have a look at the Pytorch documentation to understand what is being produced
    * Finally, examine the output of the RNN model and the output of the dataloader - what do you obserse? (The code below will help do this)
 
```python
first_batch = next(train_loader.__iter__())
print(f"First batch shape: {first_batch[1].shape}")
print(f"First batch obs:\n{first_batch[1]}")
print(f"First batch trgt:\n{first_batch[0]}")
with torch.no_grad():
    mdl_pred = model(first_batch[1])
    print(f"All hidden: {mdl_pred[0].shape}")
    print(f"All hidden values:\n {mdl_pred[0]}")
    print(f"Final hidden: {mdl_pred[1].shape}")
    print(f"Final hidden values:\n {mdl_pred[1]}")
```

In [17]:
class VanillaRNN(nn.Module):
    
    def __init__(self, input_dim:int,  hidden_dim:int, num_layers:int, 
                 fc_output_size:int, *args, **kwargs) -> None: 
        super().__init__()
        self._num_layers = num_layers
        self._hidden_dim = hidden_dim
        self.rnn = nn.RNN(
            input_size=input_dim,  hidden_size=hidden_dim,
            num_layers=num_layers, *args, **kwargs,
            batch_first=True
        )
        # Your code here - this should represent y^{<t>} = g_{2}(W_{y,a}a^{<t>} + b_{y})
        self.relu = nn.ReLU()
        self.fc = nn.Linear(in_features=hidden_dim, out_features=fc_output_size)
        # Your code here - END
    
    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out = self.rnn(x, hidden)
        # Your code here - this should represent both equations from the stanford cheatsheet
        return self.fc(out[1].squeeze())
        # Your code here - END

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self._num_layers, batch_size, self._hidden_dim).to(device)
        return hidden


In [24]:
model = VanillaRNN(input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, fc_output_size=fc_output_size)
model = model.to(device)

epochs = 5
lr = 0.0001
gpu = True
optimizer=torch.optim.Adam(model.parameters(), lr=lr)
criterion=nn.MSELoss()
epoch_train_loss, epoch_val_loss = train(
    model=model, train_data_loader=train_loader, val_data_loader=val_loader, gpu = gpu, 
    optimizer=optimizer, criterion=criterion, epochs=epochs, 
)

WandB not in use!


Running training epoch


547it [00:01, 363.21it/s]
183it [00:00, 1993.81it/s]


Running validation
Running training epoch


547it [00:01, 348.30it/s]
183it [00:00, 1333.87it/s]


...

Running validation
Running training epoch


547it [00:01, 307.72it/s]
183it [00:00, 1294.97it/s]


Running validation
Running training epoch


547it [00:01, 300.90it/s]
183it [00:00, 1378.98it/s]

Running validation


In [25]:
first_batch = next(train_loader.__iter__())
print(f"First batch shape: {first_batch[1].shape}")
print(f"First batch obs:\n{first_batch[1]}")
print(f"First batch trgt:\n{first_batch[0]}")


first_batch_inputs = first_batch[1].to(device)

with torch.no_grad():
    mdl_pred = model(first_batch_inputs) 
    print(f"All hidden: {mdl_pred[0].shape}")
    print(f"All hidden values:\n {mdl_pred[0]}")
    print(f"Final hidden: {mdl_pred[1].shape}")
    print(f"Final hidden values:\n {mdl_pred[1]}")

First batch shape: torch.Size([2, 2, 4])
First batch obs:
tensor([[[0.5731, 0.5034, 0.1042, 0.6601],
         [0.5502, 0.4516, 0.1152, 0.6482]],

        [[0.7795, 0.2595, 0.3292, 0.5059],
         [0.8062, 0.1900, 0.2742, 0.4822]]])
First batch trgt:
tensor([[24.8750],
        [31.1250]])
All hidden: torch.Size([1])
All hidden values:
 tensor([16.8468], device='cuda:0')
Final hidden: torch.Size([1])
Final hidden values:
 tensor([16.8486], device='cuda:0')


In [27]:
wandb_config={
    "lr": 0.0001,
    "hidden_dim": hidden_dim,
    "num_layers": 1,
    "fc_output_size": fc_output_size,
    "lookback": lookback,
    "nonlinearity":nonlinearity
}
model = VanillaRNN(
    input_dim=input_dim,  hidden_dim=wandb_config["hidden_dim"],
    num_layers=wandb_config["num_layers"], fc_output_size=wandb_config["fc_output_size"], 
    nonlinearity=nonlinearity
)
model = model.to(device)
with torch.no_grad():
    mdl_pred = model(first_batch[1].to(device))
    print(f"Pred y: {mdl_pred.shape}")
    print(f"True y: {first_batch[0].shape}")

optimizer=torch.optim.Adam(model.parameters(), lr=wandb_config["lr"])
criterion=nn.MSELoss()
epochs = 5
epoch_train_loss, epoch_val_loss = train(
    model=model, train_data_loader=train_loader, val_data_loader=val_loader, gpu = gpu, 
    optimizer=optimizer, criterion=criterion, epochs=epochs, wandb_proj=WANDB_PROJ,
    wandb_config=wandb_config, debug=True
)

Pred y: torch.Size([2, 1])
True y: torch.Size([2, 1])


Running training epoch


547it [00:01, 461.17it/s]
183it [00:00, 1313.38it/s]


Running validation
Running training epoch


547it [00:01, 495.44it/s]
183it [00:00, 1382.70it/s]


...

547it [00:01, 321.13it/s]
183it [00:00, 1252.32it/s]


Running validation
Running training epoch


547it [00:01, 330.51it/s]
183it [00:00, 1362.67it/s]


Running validation


train_loss,█▄▁▁▁
train_loss,45.19551
val_loss,nan


__Exercise__ 5:
* Notice under "Run summary" a "nan" is returned. The training loop provided at the beginning of this script has been augmented with the functionality to push the ground truth values and predicted values from the validation set to weights and biases. Use weights and biases to debug why nans are being produced in the validation and implement the fix.
* _Hint_:
    * Examine the validation ground truth closely - try exporting it to a csv!

The code explicitly filters out data points that are NaN in the target column (trgt_col), thus ensuring that the data input to the model is complete and has no missing values

In [30]:
non_na_idx = ~train_df[trgt_col].isna()
train_dataset = PandasTsDataset(
    X=train_df[non_na_idx][indp_cols], 
    y=train_df[non_na_idx][trgt_col],
    lookback=lookback
)
non_na_idx = ~val_df[trgt_col].isna()
val_dataset = PandasTsDataset(
    X=val_df[non_na_idx][indp_cols], 
    y=val_df[non_na_idx][trgt_col],
    lookback=lookback
)
train_loader = DataLoader(dataset=train_dataset, shuffle=shuffle, batch_size=2)
val_loader = DataLoader(dataset=val_dataset, shuffle=shuffle, batch_size=2)

In [31]:
wandb_config={
    "lr": 0.0001,
    "hidden_dim": hidden_dim,
    "num_layers": 1,
    "fc_output_size": fc_output_size,
    "lookback": lookback, 
    "nonlinearity":nonlinearity
}
model = VanillaRNN(
    input_dim=len(indp_cols),  hidden_dim=wandb_config["hidden_dim"],
    num_layers=wandb_config["num_layers"], fc_output_size=wandb_config["fc_output_size"],
    nonlinearity=nonlinearity
)
optimizer=torch.optim.Adam(model.parameters(), lr=wandb_config["lr"])
criterion=nn.MSELoss()
epochs = 5
epoch_train_loss, epoch_val_loss = train(
    model=model, train_data_loader=train_loader, val_data_loader=val_loader, gpu = gpu, 
    optimizer=optimizer, criterion=criterion, epochs=epochs, wandb_proj=WANDB_PROJ,
    wandb_config=wandb_config, debug=False
)

Running training epoch


547it [00:01, 501.62it/s]
141it [00:00, 1405.49it/s]/scratch1/NOT_BACKED_UP/xinrzhen/conda/envs/pytorch/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
183it [00:00, 1361.66it/s]


Running validation
Running training epoch


547it [00:01, 308.13it/s]
183it [00:00, 1404.29it/s]


Running validation
Running training epoch


...

547it [00:01, 293.44it/s]
183it [00:00, 1410.40it/s]


Running validation
Running training epoch


547it [00:01, 322.46it/s]
183it [00:00, 1477.13it/s]


Running validation


train_loss,█▄▁▁▁
val_loss,█▂▁▁▁
train_loss,44.07481
val_loss,29.28804


## Changing the size of the lookback
A working RNN model for one step predict has been defined. 

__Exercise__ 6a:
* Try improving performance of the model by changing the lookback size: Here, we are trying to define the 'amount' of historial time steps relevant for prediction

__Exercise__ 6b:
* Try experimenting with the output of the model. In machine learning "auxiliary loss functions" are often used to improve performance. Auxiliary loss functions assess the performance of a model to do a related task in order to increase the amount of gradient signal to pass to the model. For example, in healthcare, when developing a model to predict acute kidney injury (AKI), DeepMind assessed whether the model could predict the outcome of the lab test for AKI (https://www.nature.com/articles/s41586-019-1390-1). It might be reasonable to assume that predicting the next day values for humidity, windspeed and pressure would help in the prediction for mean temperature.
* An alternate auxiliary might be to keep meantemp as the prediction target but predict the intermediatary days as well i.e., defining an RNN of the form:

![alternative text](/cs/academic/phd3/xinrzhen/xinran/TA/figures/generic_rnn.png)

* _Hint_:
    * The first auxiliary loss will require significant modifications to pretty much all of the steps above - don't worry if you're rewriting a lot of code!
    * The second auxiliary loss only requires alterating the indexing in the PandasTsDataset function and input dimensions to the fully connected head. Alternatively, would it be better to use a specific head for each intermeditey output?
    * When validating, we are still only interested in the ability for the model to predict the temperature!
    * The hyperparameters previously discussed i.e., learning rate, epochs, batch_size and the network architecture might need to be adjusted

In [32]:
for lookback in [5,10]:
    non_na_idx = ~train_df[trgt_col].isna()
    train_dataset = PandasTsDataset(
        X=train_df[non_na_idx][indp_cols], 
        y=train_df[non_na_idx][trgt_col],
        lookback=lookback
    )
    non_na_idx = ~val_df[trgt_col].isna()
    val_dataset = PandasTsDataset(
        X=val_df[non_na_idx][indp_cols], 
        y=val_df[non_na_idx][trgt_col],
        lookback=lookback
    )
    train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=2)
    val_loader = DataLoader(dataset=val_dataset, shuffle=True, batch_size=2)
    
    wandb_config={
        "lr": 0.0001,
        "hidden_dim": hidden_dim,
        "num_layers": 1,
        "fc_output_size": 1,
        "lookback": lookback,
        "nonlinearity":nonlinearity
    }
    model = VanillaRNN(
        input_dim=len(indp_cols),  hidden_dim=wandb_config["hidden_dim"],
        num_layers=wandb_config["num_layers"], fc_output_size=wandb_config["fc_output_size"],
        nonlinearity=nonlinearity
    )
    optimizer=torch.optim.Adam(model.parameters(), lr=wandb_config["lr"])
    criterion=nn.MSELoss()
    epochs = 5
    epoch_train_loss, epoch_val_loss = train(
        model=model, train_data_loader=train_loader, val_data_loader=val_loader, gpu = gpu, 
        optimizer=optimizer, criterion=criterion, epochs=epochs, wandb_proj=WANDB_PROJ,
        wandb_config=wandb_config, debug=False
    )

Running training epoch


541it [00:01, 327.92it/s]/scratch1/NOT_BACKED_UP/xinrzhen/conda/envs/pytorch/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
546it [00:01, 320.72it/s]
181it [00:00, 1695.71it/s]


Running validation
Running training epoch


546it [00:01, 338.41it/s]
181it [00:00, 1313.98it/s]


Running validation
Running training epoch


...

543it [00:01, 516.69it/s]
179it [00:00, 1749.35it/s]


Running validation
Running training epoch


543it [00:00, 574.77it/s]
179it [00:00, 1864.00it/s]


Running validation


train_loss,█▁▁▁▁
val_loss,█▁▁▂▂
train_loss,7.40396
val_loss,12.06799


## Stacked RNNs
The nn.RNN module also contains a 'num_layers' parameter. Setting 'num_layers' to greater than 1 creates a "stacked" RNN which is depicted below (credit: https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks) 

![alternative text](/cs/academic/phd3/xinrzhen/xinran/TA/figures/rnn_stacked.png)

Stacking RNNs is similar to making MLPs deeper. One may want to stack an RNN if the raw features have different 'levels' of time dependant features as each layer extracts a non-linear relationship between the input to that layer and each layer is also recursively defined for the input sequence. For the climate example here, a stacked RNN might be required if it was hypothesised that there existed 'more complex' interactions between the four input variables than can be captured by a single non-linear layer. Furthermore, these 'more complex' interactions would have to be themselves recursive else, a deeper MLP could just be used instead to extract the time t prediction, $y_{i}$.

__Exercise__ 7:
* Experiment with different numbers of RNN layers and MLP head layers

In [33]:
class StackedRNN(nn.Module):
    
    def __init__(self, input_dim: int, hidden_dim: int, num_layers: int, fc_output_size: int, *args, **kwargs) -> None:
        super(StackedRNN, self).__init__()
        self._num_layers = num_layers
        self._hidden_dim = hidden_dim
        
        # Define a stacked RNN with multiple layers
        self.rnn = nn.RNN(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            *args,
            **kwargs
        )
        
        # Fully connected layer for output
        self.fc = nn.Linear(in_features=hidden_dim, out_features=fc_output_size)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        
        # Forward propagate through the stacked RNN
        out, hidden = self.rnn(x, hidden)  # out: (batch_size, seq_len, hidden_dim), hidden: (num_layers, batch_size, hidden_dim)
        
        # Take the output from the last time step for prediction
        out = self.fc(out[:, -1, :])  # Take the last time step's output for all batches
        
        return out

    def init_hidden(self, batch_size):
        # Initialize hidden state with zeros
        hidden = torch.zeros(self._num_layers, batch_size, self._hidden_dim).to(device)
        return hidden




In [34]:
wandb_config={
    "lr": 0.0001,
    "hidden_dim": hidden_dim,
    "num_layers": 1,
    "fc_output_size": fc_output_size,
    "lookback": lookback, 
    "nonlinearity":nonlinearity
}
model = StackedRNN(
    input_dim=len(indp_cols),  hidden_dim=wandb_config["hidden_dim"],
    num_layers=wandb_config["num_layers"], fc_output_size=wandb_config["fc_output_size"],
    nonlinearity=nonlinearity
)
optimizer=torch.optim.Adam(model.parameters(), lr=wandb_config["lr"])
criterion=nn.MSELoss()
epochs = 5
epoch_train_loss, epoch_val_loss = train(
    model=model, train_data_loader=train_loader, val_data_loader=val_loader, gpu = gpu, 
    optimizer=optimizer, criterion=criterion, epochs=epochs, wandb_proj=WANDB_PROJ,
    wandb_config=wandb_config, debug=False
)

Running training epoch


543it [00:01, 313.01it/s]
179it [00:00, 1128.48it/s]


Running validation
Running training epoch


543it [00:01, 357.36it/s]
179it [00:00, 907.82it/s]


Running validation
Running training epoch


...

543it [00:01, 320.13it/s]
179it [00:00, 929.08it/s]


Running validation
Running training epoch


543it [00:01, 277.54it/s]
179it [00:00, 887.91it/s]


Running validation


train_loss,█▁▁▁▁
val_loss,█▃▂▁▁
train_loss,7.50963
val_loss,12.89885


## Predicting n step values

__Exercise__ 8:
* Experiment with using the other target columns i.e. meantemp_5_step. When using meantemp_5_step as the target variable, we are building a model can can predict the temperature 5 days in advance. Using auxiliary losses might be useful here as one may expect that if the model can predict the next day more accurately, it should be able to predict the fifth day more accurately. However, again be careful not to use the 1 day predictions in your validation assessment!

In [35]:
class VanillaRNNMulti(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, fc_output_size, output_steps, nonlinearity='tanh'):
        super(VanillaRNNMulti, self).__init__()
        self._num_layers = num_layers
        self._hidden_dim = hidden_dim
        self.output_steps = output_steps  # Store output_steps as a class attribute

        # Initialize nn.RNN correctly without passing output_steps
        self.rnn = nn.RNN(
            input_size=input_dim,  
            hidden_size=hidden_dim, 
            num_layers=num_layers,
            nonlinearity=nonlinearity,
            batch_first=True  # Ensure that the batch dimension is the first dimension
        )

        # Define a fully connected layer for the output
        self.fc = nn.Linear(hidden_dim, fc_output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Initialize hidden state
        batch_size = x.size(0)
        hidden = torch.zeros(self._num_layers, batch_size, self._hidden_dim).to(x.device)
        
        # RNN forward pass
        out, hidden = self.rnn(x, hidden)

        # Select the output from the last time step or customize for multiple time steps
        # Here we only take the output of the last time step as an example
        out = self.fc(out[:, -1, :])
        out = self.relu(out)
        return out

In [37]:
lookback = 10  # Example lookback period, the amount of data used in the past
target_columns = ["meantemp_1_step", "meantemp_5_step"]  # Predict multiple future steps
input_columns = [col for col in train_df.columns if col not in target_columns + ["date"]]  # Exclude target columns and date

# Create datasets with multiple target columns for auxiliary loss
train_dataset = PandasTsDataset(X=train_df[input_columns], y=train_df[target_columns], lookback=lookback)
val_dataset = PandasTsDataset(X=val_df[input_columns], y=val_df[target_columns], lookback=lookback)
test_dataset = PandasTsDataset(X=test_df[input_columns], y=test_df[target_columns], lookback=lookback)

print(f"Train Dataset: {len(train_dataset)} samples")
print(f"Validation Dataset: {len(val_dataset)} samples")
print(f"Test Dataset: {len(test_dataset)} samples")

Train Dataset: 1086 samples
Validation Dataset: 358 samples
Test Dataset: 105 samples


In [38]:
# Configuration for n-step prediction
output_steps = 5  # Number of steps to predict (e.g., 1-step and 5-step)
wandb_config = {
    "lr": 0.0001,
    "hidden_dim": hidden_dim,
    "num_layers": 1,
    "fc_output_size": fc_output_size,
    "lookback": lookback,
    "nonlinearity": nonlinearity,
    "output_steps": output_steps
}

model = VanillaRNNMulti(
    input_dim=input_dim,  
    hidden_dim=wandb_config["hidden_dim"],
    num_layers=wandb_config["num_layers"], 
    fc_output_size=wandb_config["fc_output_size"],
    output_steps=wandb_config["output_steps"],  # Specify the number of output steps
    nonlinearity=nonlinearity
).to(device)

# Test the model's forward pass with dummy data
with torch.no_grad():
    mdl_pred = model(first_batch[1].to(device))
    print(f"Pred y shapes: {[p.shape for p in mdl_pred]}")
    print(f"True y: {first_batch[0].shape}")

optimizer = torch.optim.Adam(model.parameters(), lr=wandb_config["lr"])
criterion = nn.MSELoss()
epochs = 5

# Train and validate the model
epoch_train_loss, epoch_val_loss = train(
    model=model, train_data_loader=train_loader, val_data_loader=val_loader, gpu=gpu, 
    optimizer=optimizer, criterion=criterion, epochs=epochs, wandb_proj=WANDB_PROJ,
    wandb_config=wandb_config, debug=True
)


Pred y shapes: [torch.Size([1]), torch.Size([1])]
True y: torch.Size([2, 1])


Running training epoch


543it [00:01, 499.59it/s]
179it [00:00, 1999.44it/s]


Running validation
Running training epoch


543it [00:00, 697.98it/s]
179it [00:00, 2032.73it/s]


...

543it [00:01, 439.37it/s]
179it [00:00, 1894.20it/s]


Running validation
Running training epoch


543it [00:01, 488.48it/s]
179it [00:00, 2026.20it/s]


Running validation


train_loss,█▁▁▁▁
val_loss,█▁▂▁▁
train_loss,7.24605
val_loss,12.02279
